# Problema 3
Se dispone de un detector CCD de 2048 × 2048 píxeles de 15 μm. 

Determinar el campo registrado con un telescopio de focal F = 6 m y el número de exposiciones necesarias para cubrir un campo de 100×100 arcmin2 mediante un mosaico de imágenes. 

Si con este detector acoplado a un telescopio se obtienen un total de 2000 cuentas al observar una estrella durante 1000 segundos, ¿cuántas cuentas se obtendrían al observar una estrella 1 magnitud más débil y con un telescopio del doble de diámetro durante el mismo tiempo?

## Planteamiento

Dada la distancia focal, la escala de placa es $1 / F$ (en radianes por unidad de longitud).

In [23]:
import numpy as np
import astropy.units as u
from astropy.coordinates import Angle

In [24]:
## Diferentes conversiones de unidades de la escala de placa
f_dist = 6 * u.m
plate_scale = 1 / f_dist * u.rad
print('Distancia focal:', f_dist)
print('Escala de placa:', plate_scale)
print('Escala de placa:', plate_scale.to(u.rad / u.mm))
print('Escala de placa:', plate_scale.to(u.arcsec / u.mm))
print('Escala de placa:', 206265 / 6000.0, 'arcsec / mm')

Distancia focal: 6.0 m
Escala de placa: 0.16666666666666666 rad / m
Escala de placa: 0.00016666666666666666 rad / mm
Escala de placa: 34.37746770784939 arcsec / mm
Escala de placa: 34.3775 arcsec / mm


Una vez conocida la escala de placa, podemos convertir las dimensiones físicas del detector en ángulo sólido en el cielo. en este caso el detector es cuadrado, por lo que sus dos lados tienen la misma longitud.

In [25]:
# Ángulo por pixel
pix_size = 15 * u.micron
detector_size1 = 2048 * pix_size
detector_size2 = 2048 * pix_size
print('Lado del detector', detector_size1.to(u.mm))

Lado del detector 30.72 mm


In [26]:
# Calculamos en segundos de arco y minutos de arco
detector_ang1 = plate_scale * detector_size1
detector_ang2 = plate_scale * detector_size2
print(f'Ángulo subtendido por el lado del detector {detector_ang1.to(u.arcsec):.2f}')
print(f'Ángulo subtendido por el lado del detector {detector_ang1.to(u.arcmin):.2f}')

Ángulo subtendido por el lado del detector 1056.08 arcsec
Ángulo subtendido por el lado del detector 17.60 arcmin


Para ángulos pequeños, el ángulo sólido correspondiente puede calcularse como si tuviéramos una figura plana. En este caso, multiplicamos el ángulo de los lados. Damos los valores en segundos de arco cuadrados y minutos de arco cuadrados.

In [27]:
detector_solid_ang = detector_ang1 * detector_ang2
print('Ángulo sólido del detector:', detector_solid_ang.to(u.arcsec * u.arcsec))
print('Ángulo sólido del detector:', detector_solid_ang.to(u.arcmin * u.arcmin))
# 

Ángulo sólido del detector: 1115296.1122114523 arcsec2
Ángulo sólido del detector: 309.8044756142923 arcmin2


Para calcular exáctamente el número de imágenes, tendríamos que calcular en detalle las posiciones de los apuntados individuales. Como el área que queremos cubrir es grande, podemos aproximar el valor del número de imágenes por la razón de las áreas.

In [29]:
campo = 100 * u.arcmin * 100 * u.arcmin
print('Campo para cubrir',campo)
ratio = (campo / detector_solid_ang).to('')
print('Razón de ángulos sólidos', ratio)
num = int(np.ceil(ratio)) # Redondeo hacia arriba y conversión a entero
print('Número de exposiciones:', num)

Campo para cubrir 10000.0 arcmin2
Razón de ángulos sólidos 32.278423286725
Número de exposiciones: 33


El número de fotones o el número de cuentas (ignorando el bias) es proporcional al tiempo de exposición y proporcional al área colectora. El área colectora es proporcional al cuadrado del diámetro del primario.

Si de la estrella 1 recibimos $n_1 = N_1 / t$ cuentas, de la estrella 2 (1 mag más débil) recibiremos $n_2$. 

*Recordar que 1 magnitud más débil significa que $m_2 = m_1 + 1$*

$ \Delta m = m_1 - m_2 = -2.5 log (n_1 / n_2) $

de donde:

$ n_2 = n_1 10^{0.4 \Delta m} $

In [12]:
# Si de la estrella 1 recibimos N / t cuentas, de la estrella 2 (1 mag más débil) recibiremos
delta_m = -1.0
NA = 2000.0
texp = 1000
na = NA / texp
nb = na * 10**(0.4 * delta_m)

NB = nb * texp

print('Total NA', NA)
print('Total NB', NB)

Total NA 2000.0
Total NB 796.2143411069944


Si el tiempo de exposición y otros factores se mantienen constantes, entre dos telescopios la razón del número de cuentas (descontando el bias) será igual a la razón de las áreas colectoras, que en general se puede considerar proporcional al diámetro de M1.

$\frac{N_{T1}}{N_{T2}} = \frac{A_{T1}}{A_{T2}} = \left(\frac{D_{T1}}{D_{T2}}\right)^2$ 

In [13]:
# El telescopio 1 tiene el diámetro mayor
# Nuestra estrella tiene estas cuentas en el telescopio 2
NB_2 = NB
print('Estrella en telescopio 2', NB_2, 'cuentas')
D_ratio12 = 2
NB_1 = NB_2 * D_ratio12**2
print('Estrella en telescopio 1', NB_1, 'cuentas')

Estrella en telescopio 2 796.2143411069944 cuentas
Estrella en telescopio 1 3184.8573644279777 cuentas
